# Scrapear Amazon con Selenium y Scrapy

In [2]:
import re
from scrapy import Request
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Enable headless mode

## Url a procesar

In [34]:
url='https://www.amazon.ae/dp/B07MPBNK9N/ref=s9_acsd_ri_bw_c2_x_3_i?pf_rd_m=A2KKU8J8O8784X&pf_rd_s=merchandised-search-3&pf_rd_r=RQ1AJ7ZWMYNTGFYYH0Y8&pf_rd_t=101&pf_rd_p=feba8f4c-e197-49b2-8fa9-da4845519cf1&pf_rd_i=15159997031'
url='https://www.amazon.ae/Mielle-Organics-MIELLE-ENCOURGES-STRENGTHENING/dp/B07N7PK9QK/ref=zg_bs_c_beauty_d_sccl_1/257-2861550-0109024?pd_rd_w=DUZ8u&content-id=amzn1.sym.3a6b4eb3-f42b-4480-b229-3c564c506c14&pf_rd_p=3a6b4eb3-f42b-4480-b229-3c564c506c14&pf_rd_r=AY7SV8XMX45KAMV588H9&pd_rd_wg=qC9U7&pd_rd_r=b9dce8f5-add6-46e8-b699-fc152c9eede9&pd_rd_i=B07N7PK9QK&psc=1'

In [35]:
driver = webdriver.Chrome(options=chrome_options)

Exception ignored in: <function Service.__del__ at 0x000002B1B88E00E0>
Traceback (most recent call last):
  File "A:\BeOn_Repositories\amazon_jupyter\venv\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "A:\BeOn_Repositories\amazon_jupyter\venv\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "A:\BeOn_Repositories\amazon_jupyter\venv\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\lugon\AppData\Local\Programs\Python\Python311\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lugon\AppData\Local\Programs\Python\Python311\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Use

In [36]:
driver.get(url)

In [37]:
html = driver.page_source

In [38]:
driver.quit()

In [39]:
selector = Selector(text=html)

In [40]:
print(selector)

<Selector query=None data='<html lang="en-ae" class=" a-js a-aud...'>


## Extraccion de la informacion

Titulo

In [41]:
title = selector.css('#productTitle::text').get()
print(title)

        Mielle Organics MIELLE - ROSEMARY MINT, SCALP & HAIR OIL, INFUSED W/BIOTIN & ENCOURGES GROWTH, FOR DAILY USE, SCALP TREATMENT, SPLIT END CARE & SCALP & STRENGTHENING OIL       


Precio

In [42]:
price = selector.css('#newAccordionRow_0 > div:nth-child(1) > div:nth-child(1) > h5:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > span:nth-child(1) > span:nth-child(1)::text').get()
print(price)

None


Best Sellers Rank

In [43]:
category_best_sellers_rank_container = selector.css("#productDetails_detailBullets_sections1 > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(2) > span:nth-child(1)")
best_sellers_rank_element = category_best_sellers_rank_container.css("span span")
print(len(best_sellers_rank_element))

0


In [44]:
for i in best_sellers_rank_element:
    print(i)

In [45]:
best_sellers_rank_content = []
for i in best_sellers_rank_element:
    pattern = r">([^<]+)<"
    matches = re.findall(pattern, i.extract(), re.DOTALL)
    cadena = "".join(matches)
    best_sellers_rank_content.append(cadena)

In [46]:
print(best_sellers_rank_content)

[]


In [47]:
contenedor = selector.css("#productDetails_detailBullets_sections1 > tbody:nth-child(1) > tr:nth-child(3) > td:nth-child(2) > span:nth-child(1)")
print(contenedor)
print(len(contenedor))

[]
0


In [48]:
span = contenedor.css('span span').getall()
print(span)

[]


In [49]:
list = []
for i in span:
    if i != ' ' and i != '  ' and i != '':
        print(f"{i}\n")
        list.append(i)
        

In [50]:
list_clean = []
for i in list:
    pattern = r">([^<]+)<"
    matches = re.findall(pattern, i, re.DOTALL)
    cadena = "".join(matches)
    list_clean.append(cadena)

In [51]:
print(list_clean)

[]


# Capturar otros provedores del producto

In [52]:
others_sellers = selector.css('.mbcMerchantName::text').get()

In [53]:
print(others_sellers)

 Trust group LLC 


In [54]:
others_offer = selector.css('div.a-section.a-spacing-none span.a-size-small.mbcMerchantName::text').getall()

In [55]:
print(others_offer)

[' Trust group LLC ', ' ALAWAD Store DXB ', ' Amazon.ae ']


In [56]:
for i in others_offer:
    print(i)

 Trust group LLC 
 ALAWAD Store DXB 
 Amazon.ae 


In [57]:
brand_store_href = selector.css('#bylineInfo::attr(href)').get()
brand_store_link = f"https://www.amazon.ae{brand_store_href}"

In [58]:
print(brand_store_link)

https://www.amazon.ae/s/ref=bl_dp_s_web_0?ie=UTF8&search-alias=aps&field-keywords=Mielle+Organics


In [63]:
others_offer_container = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row')                                    
print(others_offer_container)

[<Selector query="descendant-or-self::div[(@class and contains(concat(' ', normalize-space(@class), ' '), ' a-box-inner ')) and (@class and contains(concat(' ', normalize-space(@class), ' '), ' a-padding-small '))]/descendant-or-self::*/span[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-declarative ')]/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-row ')]" data='<div class="a-row">              <spa...'>, <Selector query="descendant-or-self::div[(@class and contains(concat(' ', normalize-space(@class), ' '), ' a-box-inner ')) and (@class and contains(concat(' ', normalize-space(@class), ' '), ' a-padding-small '))]/descendant-or-self::*/span[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-declarative ')]/descendant-or-self::*/div[@class and contains(concat(' ', normalize-space(@class), ' '), ' a-row ')]" data='<div class="a-row">              <spa...'>, <Selector query="descendant-or-self::div[(@clas

In [66]:
others_price = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row span.a-size-medium.a-color-price::text').getall()
print(others_price)

[' AED\xa039.00 ', ' AED\xa029.00 ', ' AED\xa047.00 ']


In [70]:
amazon_price = others_offer_container[2].css('span.a-size-medium.a-color-price::text').get()
print(amazon_price)

 AED 47.00 


In [84]:
others_offer_container = selector.css('div.a-box-inner.a-padding-small span.a-declarative div.a-row')         
print(f"OTHERS OFFER:{others_offer}\n")
for i in others_offer_container:
    title_store = i.css('div.a-section.a-spacing-none span.a-size-small.mbcMerchantName::text').get()
    print(title_store)
    if 'Amazon.ae' in title_store:
        final_price = i.css('span.a-size-medium.a-color-price::text').get()
        is_transmed = True
        print(f"Encontre un Strore Transmed")

OTHERS OFFER: Amazon.ae 

 Trust group LLC 
 ALAWAD Store DXB 
 Amazon.ae 
Encontre un Strore Transmed


In [83]:
print(f"Final Price:{final_price}")

Final Price: AED 47.00 
